# Experimento: Classificador para viagens

## Acesso e análise dos dados

Vamos utilizar a biblioteca https://pandas.pydata.org/ para manipulação dos dados.

In [ ]:
import pandas as pd

In [ ]:
# leitura dos dados armazenados em um CSV
dataset = pd.read_csv('viagem.csv'); 
dataset

***
<br>
Veja como os dados carregados foram interpretados ...

In [ ]:
dataset.dtypes

***
<br>
Uma primeira análise dos dados númericos, como contagem, desvio padrão, entre outros.

In [ ]:
dataset.describe()

In [ ]:
dataset['temperatura'].mean()

***
<br>
Plot gerado com base nos dados numéricos do dataset: temperatura e humidade

In [ ]:
dataset.boxplot()

## Pré-processamento

### Enconding
Alguns classificadores não aceitam features com valores nominais, então é necessário fazer o enconding em valore binários.
É o caso do classificador que será empregado mais adiante.

In [ ]:
# pd.get_dummies converte variáveis categóricas em variáveis binárias
# drop_first=True produz apenas uma serie / coluna para atributos que possuem apenas dois valores
one_hot_dataset = pd.get_dummies(dataset,drop_first=True)
one_hot_dataset

In [ ]:
# X representa exemplos de casos (dias) e seus atributos, mas removendo o atributo classe
X = one_hot_dataset[one_hot_dataset.columns[:5]] 
X

In [ ]:
# y representa a serie dos valores classe de cada exemplo
y = one_hot_dataset['viajar_va'] 
y

### Separação em Treino e Teste

In [ ]:
# Importando biblioteca de separação de treino / teste
from sklearn.model_selection import train_test_split 
# extraindo conjunto aleatório de 70% para treino and 30% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True) 

In [ ]:
X_train

## Processamento

### Treinamento
Este treinamento produz um modelo de classificador supervisionado baseado no algoritmo de Árvores de Decisão.
Um exemplo complementar da biblioteca scikit-learn pode ser encontrado em https://scikit-learn.org/stable/modules/tree.html#classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Importando classificador
# Cria o objeto de um classificador de Árvores de Decisão
clf = DecisionTreeClassifier(criterion='entropy',min_samples_leaf=1)
# Executa o treinamento (fit)
clf = clf.fit(X_train,y_train)

Como resultado do aprendizado, obtemos a seguinte árvore

In [ ]:
from sklearn import tree
tree.plot_tree(clf,rounded=True,feature_names=X.columns,fontsize=6,class_names=['va','nao_va'],proportion=True, impurity=False, filled=True, label='none')

### Teste

In [ ]:
# Faz a predição dos valores para teste
y_pred = clf.predict(X_test)
X_test,y_pred

## Avaliação
*** 

**tp** (verdadeiros positivos): casos cassificados parar uma classe corretamente<br>
**fp** (falsos positivos): casos classificados para uma classe incorretamente<br>
**tn** (verdadeiros negativos): casos classificados corretamente como falsos<br>
**fn** (falsos negativos): casos considerados de outra classe mas deveriam ser da classe em questão<br>
<br>
**Suporte**: tp + fp, ou seja, o total de casos</br>
**Precisão**: tp / (tp + fp), ou seja, proporção dos acertos em classificar exemplos positivos <br>
**Recall**: tp / (tp + fn), ou seja, proporção de acertos entre todas as calsses<br>
**f1-score**: média harmônica entre a Precisão e o Recall<br>
**acurácia**: soma do tp de todas as classes dividido pelo número de instâncias do dataset de teste (resulta em uma percentagem).<br>

Uma ótima imagem ilustrativa sobre estas métricas pode ser encontrada na __[Wikipédia](https://pt.wikipedia.org/wiki/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o#/media/Ficheiro:Precis%C3%A3o_e_revoca%C3%A7%C3%A3o.png)__


In [ ]:
from sklearn import metrics #Importando módulo de métricas para avaliação
# Acurácia do modelo, o quanto o classificador está correto?
print("Acurácia:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

print(classification_report(y_test, y_pred, target_names=['va','nao_va']))                                                                              

In [ ]:
ConfusionMatrixDisplay.from_estimator(
    clf, X_test, y_test, display_labels=['va','nao_va'], xticks_rotation="vertical"
)
plt.tight_layout()
plt.show()